In [7]:
import sqlite3
from math import log10

# Connect to the database
conn = sqlite3.connect('../Week10/inverted_index2.db')
cur = conn.cursor()

def search(query):
    # Tokenize the query
    query_words = query.lower().split()

    # Calculate the IDF values for each query word
    idfs = {}
    for word in query_words:
        cur.execute("SELECT COUNT(DISTINCT Doc_ID) FROM word_frequencies WHERE Word_ID = (SELECT ID FROM words WHERE Word = ?)", (word,))
        df = cur.fetchone()[0]
        count = cur.execute("SELECT COUNT(*) FROM documents").fetchone()[0]
        idfs[word] = log10( count/ df)

    # Retrieve the matching documents and their TF-IDF scores
    doc_scores = {}
    for word in query_words:
        cur.execute("SELECT Doc_ID, TF_IDF FROM word_frequencies WHERE Word_ID = (SELECT ID FROM words WHERE Word = ?)", (word,))
        for row in cur.fetchall():
            doc_id, tf_idf = row
            if doc_id not in doc_scores:
                doc_scores[doc_id] = 0
            doc_scores[doc_id] += tf_idf * idfs[word]

    # Sort the matching documents by score and return the results
    results = []
    for doc_id, score in sorted(doc_scores.items(), key=lambda x: x[1], reverse=True):
        cur.execute("SELECT Link, Title FROM documents WHERE ID = ?", (doc_id,))
        link, title = cur.fetchone()
        results.append({'link': link, 'title': title,  'score': score})
    return results

# Example usage
results = search("chinese ")
print(results)
for result in results:
    print(f"{result['title']} ({result['score']:.2f}): {result['link']}")


[{'link': 'https://www.bangkokpost.com/life/social-and-lifestyle/2487324/celebrate-the-lunar-new-year-with-special-treats?utm_source=web_section&utm_medium=guru_eat_article&utm_campaign=guru', 'title': 'Celebrate the Lunar New Year with special treats', 'score': 62.343425320348246}, {'link': 'https://www.bangkokpost.com/life/social-and-lifestyle/2487324/celebrate-the-lunar-new-year-with-special-treats', 'title': 'Celebrate the Lunar New Year with special treats', 'score': 62.343425320348246}, {'link': 'https://edition.cnn.com/2023/02/06/china/china-response-suspected-spy-balloon-intl-hnk', 'title': 'Chinese balloon fallout: Beijing goes into crisis management mode - CNN', 'score': 35.62481446877043}, {'link': 'https://edition.cnn.com/2023/03/09/china/china-russia-military-defense-ties-ukraine-war-intl-hnk', 'title': "Putin's war has not changed China and Russia's deep defense sector ties | CNN", 'score': 24.9373701281393}, {'link': 'https://edition.cnn.com/2022/05/11/sport/zhou-jihong-